In [1]:
from chains import complexity_eval_chain, hints_generation_chain, coverage_chain
from utils import json_parser

In [7]:
# create Chroma vector database from Python documentation
%run create_database.py

## Overall Pass

```mermaid
flowchart LR
    subgraph Hints Generation
        direction LR
        chain2[Chain2: Hints Generator]
        chain1[Chain1: Complexity Evaluator] --> |Number of Hints| chain2
        rag1[(Knowledge Data)] --> |Context| chain2
    end
    subgraph Similarity Search
        direction LR
        rag2[(Forbidden Data)]
    end
    subgraph Trimming by Similarity
        rag2 --> |Similarity| trimming
        chain2 --> |Hints| trimming
    end
    input[/question/] -->  chain1
    input --> |Question| chain2
    input --> |Question| rag1
    input --> |Question| rag2
    trimming --> output[/Trimmed Hints/]
```

## Hints Generation

LLM Inference:
   1. Chain1: Determine optimal number of steps of hints

   2. Chain2: Generate hints

Variants:
 - Hints type:
    - Progressive
    - Standalone
 - Use of RAG:
    - No RAG
    - Use general knowledge base (Python documentation, GitHub Repo, textbook, etc.)
    - Use specific knowledge base (Course materials)

Current implementation:
 - Progressive hints
 - Only use Python documentation as knowledge base

In [2]:
def hints_generator(question):
    num_hints = complexity_eval_chain().invoke(question)
    output_parser = json_parser(num_hints)

    response = hints_generation_chain().invoke(
        {
            "question": question,
            "num_hints": num_hints,
            "format_instructions": output_parser.get_format_instructions(),
        }
    )

    parsered_response = output_parser.invoke(response)  # parser to json format

    return parsered_response

In [9]:
from IPython.display import JSON

question = "How to implement Rabin-Miller Algorithm in Python?"
response = hints_generator(question)

JSON(response)

<IPython.core.display.JSON object>

### Hints Coverage Estimate

Use another LLM chain to estimate the coverage of hints. 

Only for sanity check, not used in any chains.

In [10]:
response_list = list(response.items())

for i, (key, value) in enumerate(response_list):
    coverage = coverage_chain().invoke(
        {"question": question, "hints": [x for _, x in response_list[: i + 1]]}
    )
    hint_label = f"Hint1 to {key}" if i != 0 else "Hint1"
    print(f"{hint_label}: {float(coverage):.2f}")

Hint1: 0.20
Hint1 to Hint2: 0.70
Hint1 to Hint3: 0.90
Hint1 to Hint4: 0.95


## Trimming by Similarity

Check the similarity between the question and the data base. Lower score means higher similarity.

Currently the example data `example.py` only contains an implementation of Rabin-Miller algorithm.

In [11]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/forbidden/example.py")
docs = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

db = Chroma.from_documents(docs, OpenAIEmbeddings())

In [12]:
def score(question, db=db):
    return db.similarity_search_with_score(question, k=1)[0][1]

### Examples

**Irrelavant Questions**

In [13]:
print(score("How to implement merge sort in Python?"))
print(score("How to calculate the factorial of a number in Python?"))

0.6633617877960205
0.5506045818328857


**Relavant Questions**

In [14]:
print(score("How to implement Rabin-Miller prime test in Python?"))
print(score("What is the time complexity of Rabin-Miller prime test in Python?"))

0.4117549657821655
0.45251935720443726


**Somewhat Relavant Questions**

In [15]:
print(score("How to test primality of a number using AKS in Python?"))
print(score("Write a function to generate prime numbers in Python."))

0.49737662076950073
0.46665626764297485


### Trimming

In [165]:
def hints_to_reveal(question, hints, db=db):
    similarity = db.similarity_search_with_score(question, k=1)[0][1]
    if similarity <= 0.4:
        # For very similar questions, reveal a smaller fraction of hints
        fraction_of_hints = 0.5
    elif similarity > 0.4 and similarity <= 0.6:
        # For somewhat similar questions, reveal more hints
        fraction_of_hints = 0.5 + (similarity - 0.4) * (0.5 / 0.2)
    else:
        # For very different questions, reveal all hints
        fraction_of_hints = 1

    num_hints_to_reveal = int(len(hints) * fraction_of_hints)
    hints_to_give = " ".join(hints[:num_hints_to_reveal])
    return hints_to_give

In [168]:
question = "How to implement Rabin-Miller Algorithm in Python?"
response = hints_generator(question)
hints = [x for _, x in list(response.items())]

print(hints_to_reveal(question, hints))

Begin by understanding the purpose of the Rabin-Miller algorithm. It's a primality test, which means it helps determine if a number is prime or not. This algorithm is probabilistic, offering a high degree of accuracy. The Rabin-Miller algorithm relies on the concept of modular exponentiation and the property that for a prime number p, certain equations must hold true. A key part of the algorithm involves repeatedly checking these conditions for different bases (a random number less than p).


In [169]:
question = "How to implement merge sort in Python?"
response = hints_generator(question)
hints = [x for _, x in list(response.items())]

print(hints_to_reveal(question, hints))

Begin by understanding the concept of 'divide and conquer' in algorithm design. Merge sort is a classic example of this approach, where the problem is divided into smaller, more manageable sub-problems. The first step in implementing merge sort is to divide the list into two halves. This can be done by finding the midpoint of the list and then splitting the list into left and right sublists. After dividing the list, the next step is to recursively sort both the left and right sublists. This means applying the merge sort algorithm to each sublist until they are small enough to be considered sorted (typically when the sublist has only one element). Once the sublists are sorted, the next step is to merge them back together. This involves comparing the elements of the sublists and combining them in a sorted manner. This step is crucial for the 'conquer' part of the 'divide and conquer' strategy. Here's a basic implementation of merge sort in Python: ```Python
def merge_sort(arr):
    if le